# SPEI计算

使用`climate-indices`库直接进行计算。

<https://climate-indices.readthedocs.io/en/latest/index.html#spei-monthly>

In [30]:
import numpy as np
import pandas as pd
import xarray as xr

from climate_indices import compute, indices, utils

文档有点少，先进行测试计算

In [31]:
test_pet_frame = xr.open_dataset('../../outputs/pet-resampled/2000_01.tif').squeeze("band",drop=True)
test_precip_frame = xr.open_dataset('../../outputs/precip-cutted/_2000_2002_1.tif').squeeze("band",drop=True)

In [32]:
test_pet_frame

<xarray.Dataset>
Dimensions:      (x: 840, y: 960)
Coordinates:
  * x            (x) float64 114.0 114.0 114.0 114.0 ... 121.0 121.0 121.0 121.0
  * y            (y) float64 40.0 39.99 39.98 39.97 ... 32.03 32.02 32.01 32.0
    spatial_ref  int64 ...
Data variables:
    band_data    (y, x) float32 ...

In [33]:
test_precip_frame

<xarray.Dataset>
Dimensions:      (x: 840, y: 960)
Coordinates:
  * x            (x) float64 114.0 114.0 114.0 114.0 ... 121.0 121.0 121.0 121.0
  * y            (y) float64 40.0 39.99 39.98 39.97 ... 32.03 32.02 32.01 32.0
    spatial_ref  int64 ...
Data variables:
    band_data    (y, x) float32 ...

In [35]:
test_pet_array=test_pet_frame.band_data.to_numpy()
test_precip_array=test_precip_frame.band_data.to_numpy()*0.1

In [38]:
spei = indices.spei(test_precip_array, test_pet_array, scale=1, distribution=indices.Distribution.gamma,
                    periodicity=compute.Periodicity.monthly,data_start_year=0,calibration_year_initial=0,
                    calibration_year_final=1)



这个库文档太少，这里使用手动计算测试

## SPEI计算方法

### 收集相关数据

这里需要逐月降水$P_i$和潜在蒸散发$PET_i$的数据。

### 计算差值序列

$$ D_i=P_i-PET_i $$

### 将序列推广到时间尺度

$$ D_n^k=\sum_{i=0}^{k-1}(P_{n-i}-PET_{n-i}),\quad n\ge k $$

### 对${D_n^K}$进行正态化

通常认为该差值$D$服从三参数Log-logisitic分布。

$$ f(x)=\frac{\beta}{\alpha}\left(\frac{x-\gamma}{\alpha}\right)^{\beta-1}\left[1+\left(\frac{x-\gamma}{\alpha}\right)^{\beta}\right]^{-2}$$

其中，$\alpha,\beta,\gamma$分别为尺度、形状和起始参数。对其进行参数估计以得到相关参数。将其带入分布函数

$$ F(x)=\left[1+\left(\frac{a}{x-\gamma}\right)^{\beta}\right]^{-1} $$

得到序列的概率分布。概率分布函数带入对应$D_i$的值就得到标准化处理之前的SPEI值。

之后进行正态标准化处理，

$$ P=1-F(X) $$

带入标准化计算式，即得SPEI。
